In [ ]:
# First part of twitter project

In [1]:
from tweepy import OAuthHandler
from tweepy import API


In [2]:
consumer_key = 'IFsgWtE2fw56RITBsl5A279cq'
consumer_secret = 'kvhA9lyR6H72YN7Dxr3HwQ2Jqe6EQ9KF1oHxUpfTgZ1SokXLbt'
access_token = '1139005664125231106-1WyuyTHk3mwNS4joBU30jHhDqC60s3'
access_token_secret = 'NSwAP2E2s6G1v3CNl2a3j4fj7oLYtzkHxUqcOSjKSHrYg'

In [3]:
# Consumer key authentication
auth = OAuthHandler(consumer_key, consumer_secret)

# Access key authentication
auth.set_access_token(access_token, access_token_secret)

In [4]:
# Set up the API with the authentication handler
api = API(auth)

In [5]:
from tweepy.streaming import StreamListener
import json
import time
import sys

class SListener(StreamListener):
    def __init__(self, api = None, fprefix = 'streamer'):
        self.api = api or API()
        self.counter = 0
        self.fprefix = fprefix
        self.output  = open('%s_%s.json' % (self.fprefix, time.strftime('%Y%m%d-%H%M%S')), 'w')


    def on_data(self, data):
        if  'in_reply_to_status' in data:
            self.on_status(data)
        elif 'delete' in data:
            delete = json.loads(data)['delete']['status']
            if self.on_delete(delete['id'], delete['user_id']) is False:
                return False
        elif 'limit' in data:
            if self.on_limit(json.loads(data)['limit']['track']) is False:
                return False
        elif 'warning' in data:
            warning = json.loads(data)['warnings']
            print("WARNING: %s" % warning['message'])
            return


    def on_status(self, status):
        self.output.write(status)
        self.counter += 1
        if self.counter >= 20000:
            self.output.close()
            self.output  = open('%s_%s.json' % (self.fprefix, time.strftime('%Y%m%d-%H%M%S')), 'w')
            self.counter = 0
        return


    def on_delete(self, status_id, user_id):
        print("Delete notice")
        return


    def on_limit(self, track):
        print("WARNING: Limitation notice received, tweets missed: %d" % track)
        return


    def on_error(self, status_code):
        print('Encountered error with status code:', status_code)
        return 


    def on_timeout(self):
        print("Timeout, sleeping for 60 seconds...")
        time.sleep(60)
        return 

In [ ]:
keywords_to_track = ['#gambles', '#gamble', '#odds', '#bet', '#losses',
                    '#wager', '#lottery', '#risky', '#tradeoff', '#lucky',
                    '#random', '#casino', '#unlucky', ]